## Testing Dataset

In [1]:
import gzip
path = './poetry1.txt.gz' # we just use one of the four datasets
text = gzip.open(path, 'rt').read().lower().replace('\ufeff', ' ') # now we get the poetry

In [2]:
print('corpus length:', len(text))

corpus length: 162595


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars)) # you can print indices_char

total chars: 40


In [4]:
maxlen = 100
step = 2
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])  # print it
    next_chars.append(text[i + maxlen]) # print it and then you'll know the goal of the experience
print('nb sequences:', len(sentences))

nb sequences: 81248


In [5]:
import numpy as np
# Vectorizes the sequences with one hot encoding
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1  
## Now we get our input and label for RNN model

Vectorization...


## Model Building

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, SGD

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
print('Build model...')

RNN = LSTM  # GRU
lsize = 512
nlayers = 2
dropout = 0
impl = 2

model = Sequential()
if nlayers == 1:
    model.add(RNN(lsize, input_shape=(maxlen, len(chars)), implementation=impl, recurrent_dropout=dropout))
else:
    model.add(
        RNN(lsize, input_shape=(maxlen, len(chars)), implementation=impl, recurrent_dropout=dropout, return_sequences=True))
    for i in range(1, nlayers - 1):
        model.add(RNN(lsize, implementation=impl, recurrent_dropout=dropout, return_sequences=True))
    model.add(RNN(lsize, implementation=impl, recurrent_dropout=dropout))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


## Training

In [8]:
# optimizer = RMSprop(lr=0.01)
#optimizer = SGD(lr=0.05, momentum=0.95)
model.compile(loss='categorical_crossentropy', optimizer="adam")

bsize = 2048
iterations = 15
epoch_it = 10

# File for saving the generated text each iteration
gfile = open('tgenerated-F%s-ML%d-S%d-NL%d-D%3.2f-BS%d.txt' % (path.split('.')[0], maxlen, lsize, nlayers, dropout, bsize), 'w')

In [11]:
def sample(preds, temperature=1.0):
    """
    helper function to sample an index from a probability array

    :param preds:
    :param temperature:
    :return:
    """

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
def generate_text(seed, numlines, gfile, wseed=False):
    """
    Generates a number of lines (or at most 1000 characters) using the given seed
    :param seed:
    :param lines:
    :return:
    """
    generated = ''
    gprinted = ''
    sentence = seed
    generated += sentence

    nlines = 0
    for i in range(1000):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        gprinted += next_char

        sentence = sentence[1:] + next_char
        # Count the number of lines generated
        if next_char == '\n':
            nlines += 1
        if nlines > numlines:
            break

    if wseed:
        gfile.write(seed + gprinted)
    else:
        gfile.write(gprinted)
    gfile.write('\n')
    gfile.flush()

In [13]:
# train the model, output generated text after each iteration
for iteration in range(iterations):
    print()
    print('-' * 50)
    print('Iteration %d' %((iteration + 1)))
    model.fit(X, y,
              batch_size=bsize,
              epochs=epoch_it,
              verbose=1)

    gfile.write('-' * 50)
    gfile.write('\n')
    gfile.write('Iteration %d n' % (iteration + 1))
    seed = random_seed(chars, maxlen)
    for diversity in [0.2, 0.4, 0.8, 1.0]:
        gfile.write('\n\n')
        gfile.write('DIV = %3.2f\n\n' % diversity)
        generate_text(seed, numlines=10, gfile=gfile, wseed=False)
        model.save(f"./textgen-T4-N{lsize}-L{nlayers}-D{dropout}-{time.strftime('%Y%m%d%H%M%S')}.h5")
gfile.close()
model.save(f"./textgen-T4-N{lsize}-L{nlayers}-D{dropout}-{time.strftime('%Y%m%d%H%M%S')}.h5")
print()
print("Ending")

# Note: If there is a ERROR of OOM(out of memory), you should use a more powerful machine.


--------------------------------------------------
Iteration 1
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[2048,512] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: lstm_1/while/clip_by_value_1/Minimum = Minimum[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](lstm_1/while/add_3, lstm_1/while/Const_5)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'lstm_1/while/clip_by_value_1/Minimum', defined at:
  File "E:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\anaconda\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "E:\anaconda\lib\site-packages\ipykernel\ke